<a href="https://colab.research.google.com/github/gmd42/LSS_4DoF_Arm_Python/blob/master/Bending_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.integrate import quad

We aim to parameterize our gripper, such that a gripper can be printed for any stent while maintaining the same performance (i.e. stiffness). For a gripper to be functional at grabbing a particular stent, it must be able to deform around the stent solely on the force of the arm moving to the stent location (i.e. not be too stiff), and it must only ever deform elastically.

To ensure that the gripper neither exceeds yield stress nor requires more force than the arm can provide, we perform the following analysis:

The purpose of this document is to calculate the forces on our gripper due to deflection. The current solution was found by trial and error because of a lack of material properties. We will analyze the geometry of our gripper using assumed (and most likely incorrect) material properties to find a force. It is acceptable if this force is incorrect because we need to determine how changing the geometry will affect the force relatively in order to make a scalable model.

Since the stent is stiff relative to the gripper, calculating the maximum deflection of each gripper half is trivial. As shown in the CAD below, the distance between the prongs of the gripper at rest is approximately 21.5 mm. Knowing the diameter of the stent, we can calculate the maximum deflection. This calculation is performed below.

<img src="https://drive.google.com/uc?id=1UiiMws1x3WFSPSME97T14JdcVkJ8Y5qG" width="500">


In [ ]:
#Constants#
Stent_OD = 29         #mm
Prong_Dist = 21.534   #mm

#Calculations#
Deflection = (Stent_OD - Prong_Dist) / 2

#Results#
print("Deflection:", round(Deflection,3), "mm")


Deflection: 3.733 mm


The images below show the dimensions of our gripper design that will be used for the remaining calculations.
<img src="https://drive.google.com/uc?id=1kpQq373apVLTn1kDsPKlB14u-UWsXuzg" width="500">

<img src="https://drive.google.com/uc?id=1j_xKJK_wsNtNdSsFDUEYVX22OZuZY_HY" width="500">



In with this design we have a curved beam coupled with a straight beam. When a beam is curved, the neutral axis does not coincide witht he centroidal axis. Our gripper has a rectangular cross section. Table 3-4 in Shigley shows this distance.

<img src="https://drive.google.com/uc?id=1BIkTKmT1CGwmzh2t2qQQGsj4C23Tudjk" width="600">



In [ ]:
#Constants#
h = 2         #mm
ri = 14.25    #mm
ro = 16.25    #mm

#Calculations#
rc = ri + (h/2)
rn = h/(np.log(ro/ri))
e = rc - rn

#Results#
print("Centroidal Radius:",round(rc,3), "mm")
print("Neutral Axis Radius:", round(rn,3), "mm")
print("Eccentricity:", round(e,3), "mm")

Centroidal Radius: 15.25 mm
Neutral Axis Radius: 15.228 mm
Eccentricity: 0.022 mm


To calculate the force required for our deflection of this complex beam, we are going to utilize Castigliano's theorem. According to Shigley, Castigliano's theroem states that:

*When forces act on elastic systems subject to small displacements, the displacement corresponding to any force, in the direction of the force, is equal to the partial derivative of the total strain energy with respect to that force.*

$\delta_i= \frac{\partial U}{\partial F_i}$

Where $\delta_i$ is the is the displacement of the point of application of the force $F_i$ in the direction of $F_i$.

For a straight beam that is uniform along it's length the following equations define the total energy of the beam.

(Shigley 4-22/23) $U_{bend}=\int\frac{M^2}{2EI}dx=\frac{M^2l}{2EI}$

(Shigley 4-24/25) $U_{shear}=\int\frac{CV^2}{2AG}dx=\frac{CV^2l}{2AG}$

Where $C$ is a correction factor from table 4-1 and is 1.2 for a rectangular cross section.

$U=U_{bend}+U_{shear}$

Castigliano's theorem can also be applied to bent beams such as the one shown below taken from Shigley.

<img src="https://drive.google.com/uc?id=1zv81E-NnIf1NouGYhs0VoCjHwmeCg1E5" width="600">


When a beam is bent, the bending moment and axial force are coupled, creating multiple energy terms. 

(Shigley 4-32)  $U_1=\int\frac{M^2d\theta}{2AeE}$

Where $e$ is the eccentricity calculated above.

(Shigley 4-34)  $U_2=\int\frac{F_\theta^2Rd\theta}{2AE}$

(Shigley 4-35)  $U_3=-\int\frac{MF_\theta d\theta}{AE}$

(Shigley 4-36)  $U_4=\int\frac{CF_r^2Rd\theta}{2AG}$

Where $C$ is a correction factor from table 4-1 and is 1.2 for a rectangular cross section.

These equations combine to form

(Shigley 4-38)

$\delta=
\frac{\partial U}{\partial F}=
\int\frac{M}{AeE}(\frac{\partial M}{\partial F})d\theta +
\int\frac{F_\theta R}{AE}(\frac{\partial F_\theta}{\partial F})d\theta - 
\int\frac{1}{AE}(\frac{\partial (MF_\theta}{\partial F})d\theta +
\int\frac{CF_rR}{AG}(\frac{\partial Fr}{\partial F})d\theta$

This equation applies to any section of a thick walled circular curved beam.

Great simplifications can be made if the beam is thin walled. This is typically defined as R/h > 10.

In [ ]:
#Calculations#
Thickness = rn/h

#Results#
print("R/h:", round(Thickness, 3))

R/h: 7.614


Unfortuantely for us this is not the case. We are close to the limit so we could probably simplify the equations, but for the sake of correctness, we will not.

The image below shows the FBD of one of our prongs. Our goal is to solve for F using the known dispacement. For simplicity, the ball at the end of the prong will be ignored.

<img src="https://drive.google.com/uc?id=1qTLQ_JIZxyn-ypPoruDfkmjwaKlxJiY_" width="650">

Our beam is comprised of two distinct parts, the straight part and the curved part. Slicing the beam along the straight part yields the following FBD.

<img src="https://drive.google.com/uc?id=1EVJz8ttxH9A1M_VI0a3Qdad0YJAkiJ4_" width="600">



$M_{AB}=Fl_{AB}$

From Shigley 4-22

$U_{bend}=\frac{F^2l_{AB}^3}{2EI}$

$C = 1.2$

$V_{AB} = F$

From Shigley 4-24

$U_{shear}=\frac{1.2F^2l_{AB}}{2AG}$

$U_{straight}=\frac{F^2l_{AB}^3}{2EI}+\frac{1.2F^2l_{AB}}{2AG}$

$\frac{\partial U}{\partial F}=\frac{Fl_{AB}^3}{EI}+\frac{1.2Fl_{AB}}{AG}=\delta_{straight}$


Slicing the beam along the curved part yields the following FBD.

<img src="https://drive.google.com/uc?id=1kWqMp0gPWkuV9MP4ozcQ9hZyEtAjhL-D" width="600">

Where $\theta$ is the angle between the vertical and $V_{BC}$

$M_{BC}=F(R_c\sin(30)+R_c\sin(\theta))=FR_c(\sin(30)+\sin(\theta))$

Where $R_c$ is the Centroidal Radius.

$V_{BC}=F\cos(\theta)$

$N_{BC}=F\sin(\theta)$

$U_1=\int_{-30}^{71.2}\frac{M_{BC}^2d\theta}{2AeE} =
\int_{-30}^{71.2}\frac{(F^2R_c^2(\sin(30)+\sin(\theta))^2}{2AeE}d\theta=
\frac{F^2R_c^2}{2AeE}\int_{-30}^{71.2}\sin^2(\theta)+2\sin(30)\sin(\theta)+\sin^2(30)d\theta$

$U_2=\int_{-30}^{71.2}\frac{N_{BC}^2R_cd\theta}{2AE}=
\int_{-30}^{71.2}\frac{F^2\sin^2(\theta)R_c}{2AE}d\theta=
\frac{F^2R_c}{2AE}\int_{-30}^{71.2}\sin^2(\theta)d\theta $

$U_3=-\int_{-30}^{71.2}\frac{M_{BC}N_{BC}}{AE} d\theta=
-\int_{-30}^{71.2}\frac{F^2R_c(\sin(30)+\sin(\theta))\sin(\theta)}{AE} d\theta=
\frac{F^2R_c}{AE}\int_{-30}^{71.2}\sin^2(\theta)+\sin(30)\sin(\theta)d\theta$

$U_4=\int_{-30}^{71.2}\frac{CV_{BC}^2R_c}{2AG}d\theta=
\int_{-30}^{71.2}\frac{1.2F^2\cos^2(\theta)R_c}{2AG}d\theta=
\frac{1.2F^2R_c}{2AG}\int_{-30}^{71.2}cos^2(\theta)d\theta$

$\delta_{curved}=\frac{\partial U}{\partial F}=(\frac{FR_C^2}{AeE}\int_{-30}^{71.2}\sin^2(\theta)+2\sin(30)\sin(\theta)+\sin^2(30)d\theta)+(\frac{FR_c}{AE}\int_{-30}^{71.2}\sin^2(\theta)d\theta)-(\frac{2FR_c}{AE}\int_{-30}^{71.2}\sin^2(\theta)+\sin(30)\sin(\theta)d\theta)+(\frac{1.2FR_c}{AG}\int_{-30}^{71.2}cos^2(\theta)d\theta)$

$\delta_{total}=\delta_{curved}+\delta_{straight}$

Factor out $F$ and divide by $\delta_{total}$

$\frac{1}{F}=\frac{l_{AB}^3}{EI\delta_{total}}+\frac{1.2l_{AB}}{AG\delta_{total}}+
(\frac{R_C^2}{AeE\delta_{total}}\int_{-30}^{71.2}\sin^2(\theta)+2\sin(30)\sin(\theta)+\sin^2(30)d\theta)+
(\frac{R_c}{AE\delta_{total}}\int_{-30}^{71.2}\sin^2(\theta)d\theta)-
(\frac{2R_c}{AE\delta_{total}}\int_{-30}^{71.2}\sin^2(\theta)+\sin(30)\sin(\theta)d\theta)+(\frac{1.2R_c}{AG\delta_{total}}\int_{-30}^{71.2}cos^2(\theta)d\theta)$

In [ ]:
#Integrals#
def u1(x):
  return np.sin(x)**2 + 2*np.sin(x)*np.sin(np.radians(30))+np.sin(np.radians(30))**2

def u2(x):
  return np.sin(x)**2

def u3(x):
  return np.sin(x)**2 + np.sin(x)*np.sin(np.radians(30))

def u4(x):
  return np.cos(x)**2

#Calcualtions#
U1, err1 = quad(u1, np.radians(-30), np.radians(71.2))
U2, err2 = quad(u2, np.radians(-30), np.radians(71.2))
U3, err3 = quad(u3, np.radians(-30), np.radians(71.2))
U4, err4 = quad(u4, np.radians(-30), np.radians(71.2))

#Results#
print("Definite of U1:", U1)
print("Definite of U2:", U2)
print("Definite of U3:", U3)
print("Definite of U4:", U4)

-0.5235987755982988
Definite of U1: 1.499421968896197
Definite of U2: 0.514093959587704
Definite of U3: 0.7859738138646678
Definite of U4: 1.2521792434305574


by solving the definite integrals, we can simplify to

$\frac{1}{F}=\frac{l_{AB}^3}{EI\delta_{total}}+\frac{1.2l_{AB}}{AG\delta_{total}}+
1.4994*\frac{R_C^2}{AeE\delta_{total}}+
0.5141*\frac{R_c}{AE\delta_{total}}-
0.7860*\frac{2R_c}{AE\delta_{total}}+
1.2522*\frac{1.2R_c}{AG\delta_{total}}$

We have now solved for force F. To make this equation useful for the parameterization of our gripper, we assess our knowns and unknowns:

Material Properties:
* resin Young's Modulus $E$
* resin Shear Modulus $G$

Gripper Properties:
* Beam length $ l $   
* Beam thickness $h$
* Beam width $b$
* Claw angle $\theta$
* Centroidal radius $R_c$
* Properties derived from $b$ and $h$
  * Moment of inertia $I$
  * Cross-sectional area $A$ 

Other:
* Force applied by stent $ F $
* Tip displacement $\delta_{total}$

From here, we can essentially check off the list of parameters until we have fully constrained our design. First, we acknowledge that $\theta$ should be the same for all stents, so the current theta value of 60 degrees will be held constant. We also intend to measure and apply constant values for material properties E and G. We will measure E and G using a cantilever setup and a thin beam, which will be printed in a similar orientation to our gripper.

So on our checklist, $E$, $G$, and $\theta$ have been crossed off, leaving  $F$, $\delta_{total}$, $l$, $h$, $b$, $R_c$, $I$, and $A$.

We want the user to be able to manufacture a gripper for any stent size, so we consider stent's outer diameter as being known. Some parameters are functions of stent diameter:

$R_c = ( D_{stent} + h ) * 0.5$

$\delta_{total} = ( R_c - h * 0.5 ) * cos(\theta) - h  $  
By substiting for $R_c$ and simplifying, we get  
$\delta_{total} = D_{stent} * 0.5 * cos(\theta) - h$

We also note that $I$ and $A$ are directly derived from $h$ and $b$ by the following equations:

$ A = h * b $

$ I = \frac{1}{12}bh^3 $

Now, we've reduced our checklist of parameters down to $F$, $l$, $h$, and $b$.

We recall the following from our progress so far on the gripper and the arm as a whole:
* a gripper with an inadequate $b$ value relative to the length of the stent was unstable
* changing the value of *l* would require adjusting the code for the arm, which shouldn't have to change for any stent size

*************************

We really want the ceter of the stent to be the same, not l

Based on this information, l should be held completely constant:

$ l = 20mm $


************************
And since we have a functioning value for $b/D_{stent}$, we may allow the following equation to define $b$:

$ \frac{b_{current}}{D_{stent_{current}}} = \frac{b}{D_{stent}} $

$\therefore b = \frac{b_{current} * D_{stent}}{D_{stent_{current}}} =
\frac{12.00}{29.00} * D_{stent} = 0.4138 * D_{stent}$

Now our only unknowns are $F$ and $h$. In other words, for a given diameter, we can reduce the original function to a much simpler $F(h, D_{stent}) $

$\frac{1}{F}=\frac{l_{AB}^3}{EI\delta_{total}}+\frac{1.2l_{AB}}{AG\delta_{total}}+
\frac{R_C^2}{AeE\delta_{total}}* 1.4994 +
\frac{R_c}{AE\delta_{total}}*0.5141-
\frac{2R_c}{AE\delta_{total}}*1.0579 +
\frac{1.2R_c}{AG\delta_{total}}*1.252$

